In [2]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from model import llm
from langchain_core.messages import HumanMessage, AIMessage

USER_AGENT environment variable not set, consider setting it to identify your requests.
2024-07-02 11:18:53.729 
  command:

    streamlit run /home/duclv/.conda/envs/mm/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [3]:

docs = TextLoader('/mnt/HDD1/duclv/ACM-AMM/database/uploaded_docs/Earthquakes Conversation.txt').load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
with open("/mnt/HDD1/duclv/open_ai_key.txt", "r") as f:
    openai_api_key = f.read().strip()
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(openai_api_key=openai_api_key))

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")



def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)
from langchain_core.prompts import PromptTemplate
prompt = """
SYSTEM 
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Context: {context} 
Chat history: {chat_history}
HUMAN

Question: {question} 

Answer:"""

prompt_template = PromptTemplate.from_template(prompt)

chain = prompt_template | llm


In [4]:
chat_history = []

question = f"who is speaker_00 "
ai_msg_1 = chain.invoke({"question": question, "chat_history": chat_history, "context": format_docs(splits)}).content
print(ai_msg_1)
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1)])

second_question = "what is the topic of the conversation?"
ai_msg_2 = chain.invoke({"question": second_question, "chat_history": chat_history, "context": format_docs(splits)}).content
chat_history.extend([HumanMessage(content=second_question), AIMessage(content=ai_msg_2)])
print(ai_msg_2)

third_question = "what is the topic of the conversation?"
ai_msg_2 = chain.invoke({"question": second_question, "chat_history": chat_history, "context": format_docs(splits)}).content
chat_history.extend([HumanMessage(content=second_question), AIMessage(content=ai_msg_2)])
print(ai_msg_2)

Based on the provided transcript, Speaker_00 is Joe Weiss.
The topic of the conversation is earthquakes and personal experiences with them. The speakers share their stories about feeling earthquakes in different locations, including California, Bangkok, and Japan. They discuss how they initially thought the shaking was caused by something else (like a train or Mack truck), and how it's common to not realize an earthquake has occurred until later.
The topic of the conversation is earthquakes and personal experiences with them. The speakers share their stories about feeling earthquakes in different locations, including California, Bangkok, and Japan. They discuss how they initially thought the shaking was caused by something else (like a train or Mack truck), and how it's common to not realize an earthquake has occurred until later.


In [5]:
chat_history = []
while question!="exit":
    question = input("Question: ")
    ai_msg = chain.invoke({"question": question, "chat_history": chat_history, "context": format_docs(splits)}).content
    chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg)])
    print(ai_msg)
    print("Chat history: ", chat_history)
    # print("Context: ", format_docs(splits))
    print("Question: ", question)
    print("Answer: ", ai_msg   )

Xin chào! (Hello!) It seems like you're trying to start a conversation or ask a question. However, I noticed that the provided text appears to be a transcript of a conversation between two speakers, SPEAKER_01 and SPEAKER_00. If you'd like to ask a specific question related to this conversation, feel free to do so!
Chat history:  [HumanMessage(content='xin chào'), AIMessage(content="Xin chào! (Hello!) It seems like you're trying to start a conversation or ask a question. However, I noticed that the provided text appears to be a transcript of a conversation between two speakers, SPEAKER_01 and SPEAKER_00. If you'd like to ask a specific question related to this conversation, feel free to do so!")]
Question:  xin chào
Answer:  Xin chào! (Hello!) It seems like you're trying to start a conversation or ask a question. However, I noticed that the provided text appears to be a transcript of a conversation between two speakers, SPEAKER_01 and SPEAKER_00. If you'd like to ask a specific questio

In [6]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [7]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks and write meeting minutes tasks \
Use the following context to answer question or writing the meeting minutes\
The answer or meeting minutes should be based on the below context. \
If the question is not answerable based on the context, you can answer with 'I don't know'.\
If user ask question, answer the question in concise way.\
if use ask to write meeting minutes, write the meeting minutes based on the context. \ 

CONTEXT: {context}\
"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [8]:
context  = format_docs(docs)
context

"from 3.294 to 4.095\tSPEAKER_00: Hey, check this out.\nfrom 4.475 to 6.597\tSPEAKER_00:You know what Eric asked me when I got into work this morning?\nfrom 6.617 to 8.339\tSPEAKER_01:I have no idea.\nfrom 8.359 to 9.28\tSPEAKER_01:What did he ask you?\nfrom 9.88 to 11.942\tSPEAKER_00:He asked me if I felt the earthquake last night.\nfrom 13.224 to 13.984\tSPEAKER_01:Earthquake?\nfrom 14.405 to 15.526\tSPEAKER_01:You've got to be kidding.\nfrom 15.566 to 16.727\tSPEAKER_01:I didn't feel an earthquake.\nfrom 16.987 to 18.008\tSPEAKER_00:I know, that's what I said.\nfrom 18.028 to 20.571\tSPEAKER_00:He told me it actually woke him up last night.\nfrom 21.581 to 27.562\tSPEAKER_00: You know, yeah, it was like it went, it happened at like, uh, four 42 in the morning.\nfrom 27.742 to 38.124\tSPEAKER_00:So we must've been sleeping, but I mean, it's possible that, you know, even if we had been awake, we might not have felt it because you know, maybe it, uh, wasn't felt, you know, this far Nor

In [9]:
import langchain.document_loaders
from langchain_core.messages import HumanMessage, AIMessage
import langchain
langchain.debug = False
chat_history = []

question = f"xin chào "
ai_msg_1 = question_answer_chain.invoke({"input": question, "chat_history": chat_history, "context": docs})
print(ai_msg_1)
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1)])

second_question = "what is the topic of the conversation?"
ai_msg_2 = question_answer_chain.invoke({"input": second_question, "chat_history": chat_history, "context": docs})


chat_history.extend([HumanMessage(content=second_question), AIMessage(content=ai_msg_2)])
print(ai_msg_2)

Xin chào! (Hello!) How can I help you today? Do you need assistance with a question, meeting minutes, or something else?
The topic of the conversation appears to be earthquakes and people's experiences with them, including their reactions and perceptions during and after the events. The speakers share personal anecdotes about feeling earthquakes in different locations (Bangkok, Japan, and California) and discuss how they initially interpreted the sensations as something other than an earthquake (e.g., a train or a Mack truck).


In [10]:
import langchain
langchain.debug = True
third_question = "who is speaker_00 in this conversation?"
ai_msg_3 = question_answer_chain.invoke({"input": third_question, "chat_history": chat_history, "context": docs})

print(ai_msg_3)

[chain/start] [chain:stuff_documents_chain] Entering Chain run with input:
[inputs]
[chain/start] [chain:stuff_documents_chain > chain:format_inputs] Entering Chain run with input:
[inputs]
[chain/start] [chain:stuff_documents_chain > chain:format_inputs > chain:RunnableParallel<context>] Entering Chain run with input:
[inputs]
[chain/start] [chain:stuff_documents_chain > chain:format_inputs > chain:RunnableParallel<context> > chain:format_docs] Entering Chain run with input:
[inputs]
[chain/end] [chain:stuff_documents_chain > chain:format_inputs > chain:RunnableParallel<context> > chain:format_docs] s] Exiting Chain run with output:
{
  "output": "from 3.294 to 4.095\tSPEAKER_00: Hey, check this out.\nfrom 4.475 to 6.597\tSPEAKER_00:You know what Eric asked me when I got into work this morning?\nfrom 6.617 to 8.339\tSPEAKER_01:I have no idea.\nfrom 8.359 to 9.28\tSPEAKER_01:What did he ask you?\nfrom 9.88 to 11.942\tSPEAKER_00:He asked me if I felt the earthquake last night.\nfrom 13.

[llm/end] [chain:stuff_documents_chain > llm:ChatOllama] [3.24s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Based on the provided transcript, Speaker_00 appears to be one of the two individuals participating in the conversation. The other individual is referred to as Speaker_01.\n\nIt seems that Speaker_00 is sharing their personal experiences and perspectives on earthquakes, including their own experiences living in California and Japan. They also mention specific events, such as the Loma Prieta earthquake, and share stories about how they felt during those events.\n\nSpeaker_00's role in the conversation appears to be one of storytelling and sharing their thoughts and feelings about earthquakes.",
        "generation_info": {
          "model": "llama3",
          "created_at": "2024-07-02T03:21:44.218064263Z",
          "message": {
            "role": "assistant",
            "content": ""
          },
          "done_reason": "stop",
         

In [11]:
chat_history

[HumanMessage(content='xin chào '),
 AIMessage(content='Xin chào! (Hello!) How can I help you today? Do you need assistance with a question, meeting minutes, or something else?'),
 HumanMessage(content='what is the topic of the conversation?'),
 AIMessage(content="The topic of the conversation appears to be earthquakes and people's experiences with them, including their reactions and perceptions during and after the events. The speakers share personal anecdotes about feeling earthquakes in different locations (Bangkok, Japan, and California) and discuss how they initially interpreted the sensations as something other than an earthquake (e.g., a train or a Mack truck).")]